In [ ]:
import fastbook
fastbook.setup_book()

installing and importing the fastbook package.

In [ ]:
#hide
from fastbook import *
from fastai.vision.widgets import *

The required libraries from the fastai package are imported. fastai.vision.widgets includes tools to interact with the vision models such as image uploaders and predictions display.

In [ ]:
search_images_ddg

In [ ]:
face_problem_types = ['DARK CIRCLES ON FACE','DARK SPOTS ON FACE','EYE BAGS','DRY SKIN','OILY SKIN','FACE REDNESS',
          'DULL SKIN','ROUGH TEXTURE ON FACE','LARGE PORES ON FACE','UNDER-EYE WRINKLES','HORMONAL ACNE',
          'BLACKHEADS','SEBACEOUS FILAMENTS','FOREHEAD WRINKLES','RAZOR BUMPS']
path = Path('Skin_Problems')

- A list of common skin problems is defined. This list will be used to search for images of these problems online.
- A directory Skin_Problems is created to store images related to these skin issues.

In [ ]:
if not path.exists():
    path.mkdir()
    for face_problem_type in face_problem_types:
        dest = (path/face_problem_type)
        dest.mkdir(exist_ok=True)
        results = search_images_ddg(face_problem_type,max_images=500)
        download_images(dest, urls=results)

The code checks if the directory exists. If not, it creates a directory for each skin problem and downloads images using DuckDuckGo’s image search API. This forms the dataset for training the model.

In [ ]:
fns = get_image_files(path)
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink);

The downloaded images are verified to ensure they are valid. Any corrupted or invalid images are removed to clean the dataset.

In [ ]:
face_problems = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.1, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

The DataBlock is defined, which is the core data structure in fastai for managing data transformations and pipelines. It specifies how to split the data, how to label it, and what transformations to apply (like resizing images).

In [ ]:
dls = face_problems.dataloaders(path)

The DataBlock is used to create data loaders, which handle the loading and transformation of images during training.

In [ ]:
face_problems = face_problems.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=2))
dls = face_problems.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=True)

In [ ]:
face_problems = face_problems.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms(),
)
dls = face_problems.dataloaders(path,bs=16)

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=error_rate)
learn.fine_tune(5)

Data augmentation is applied to improve model robustness. The model is then trained using a pre-trained ResNet50 architecture with a few epochs of fine-tuning.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(1, nrows=1)

The trained model is evaluated using confusion matrices and loss plots to analyze performance and identify areas where the model might be failing.

In [ ]:
learn.export()

The trained model is exported for later use. This allows the model to be loaded and used for inference without retraining.